<a href="https://colab.research.google.com/github/adc257/AmEx-Project/blob/Ye_branch/LSTM_implementation__Noise_DE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM Implementation

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import TextClassificationPipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.optim.lr_scheduler import MultiStepLR
from tqdm import tqdm

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data

In [9]:
# Import train and test data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Separate train labels and text
train_labels = train['category']
train_text = train['text']
train_labels_list = train_labels.tolist()

# Separate test labels and text
test_labels = test['category']
test_text = test['text']
test_labels_list = test_labels.tolist()

### Randomly select 10% of the dataset to be noised

In [10]:
# Set the random seed for reproducibility
np.random.seed(42)

# Calculate the number of samples for 10% of the dataset
sample_size = int(len(train) * 0.1)

# Conduct sampling
sampled_data = train.sample(n=sample_size, replace=False)

sampled_data

,text,category
6883,Is it possible for me to change my PIN number?,change_pin
5836,I'm not sure why my card didn't work,declined_card_payment
8601,I don't think my top up worked,top_up_failed
2545,Can you explain why my payment was charged a fee?,card_payment_fee_charged
8697,How long does a transfer from a UK account tak...,balance_not_updated_after_bank_transfer
...,...,...
7089,I have been trying to exchange this for crypto...,beneficiary_not_allowed
4238,I want to transfer money using my credit card.,topping_up_by_card
4901,I can't remember what my password is,passcode_forgotten
3458,My cheque is taking a while,balance_not_updated_after_cheque_or_cash_deposit


### Shuffle 10% of Labels in Dn to Create Noise

In [11]:
# Randomly sample 10% of the rows from the Dn
sampled_rows = sampled_data.sample(frac=0.1, random_state=42)

# Add new column 'k' and initialize with 0
sampled_data['k'] = 0

# Store the original labels ('l') before shuffling the 'category' column
sampled_data['l'] = sampled_data['category']  # Initialize 'l' column with original category values
sampled_data.loc[sampled_rows.index, 'l'] = sampled_data.loc[sampled_rows.index, 'category']

# Convert the selected rows' 'category' column to a numpy array
sampled_category_values = np.array(sampled_rows['category'])

# Shuffle the array
np.random.shuffle(sampled_category_values)

# Update the 'category' column with the shuffled values in the original DataFrame
sampled_data.loc[sampled_rows.index, 'category'] = sampled_category_values

# Update 'k' to 1 for the randomly selected 10% of rows
sampled_data.loc[sampled_rows.index, 'k'] = 1

# Update 'l\'' for the shuffled rows
sampled_data.loc[sampled_rows.index, 'l\''] = sampled_category_values

# Display the DataFrame to verify the changes
sampled_data.head(25)


,text,category,k,l,l'
6883,Is it possible for me to change my PIN number?,change_pin,0,change_pin,NaN
5836,I'm not sure why my card didn't work,declined_card_payment,0,declined_card_payment,NaN
8601,I don't think my top up worked,top_up_failed,0,top_up_failed,NaN
2545,Can you explain why my payment was charged a fee?,card_payment_fee_charged,0,card_payment_fee_charged,NaN
8697,How long does a transfer from a UK account tak...,balance_not_updated_after_bank_transfer,0,balance_not_updated_after_bank_transfer,NaN
5573,Why am I getting declines when trying to make ...,declined_transfer,0,declined_transfer,NaN
576,What is the $1 transaction on my account?,extra_charge_on_statement,0,extra_charge_on_statement,NaN
6832,It looks like my card payment was sent back.,reverted_card_payment?,0,reverted_card_payment?,NaN
7111,Why am I unable to transfer money when I was a...,beneficiary_not_allowed,0,beneficiary_not_allowed,NaN
439,What if there is an error on the exchange rate?,card_payment_wrong_exchange_rate,0,card_payment_wrong_exchange_rate,NaN


## Step 1: Define and Train Deep Model

In [12]:
device_name = 'cuda'
model_name = 'bert-base-uncased'

# Input Text
input_text = np.array(sampled_data['text'])
labels = np.array(sampled_data['category'])

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# Define labels
unique_labels = set(label for label in labels)
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

# Tokenize input text
input_ids = tokenizer.batch_encode_plus(input_text, padding=True, truncation=True, return_tensors='pt')['input_ids']
attention_mask = tokenizer.batch_encode_plus(input_text, padding=True, truncation=True, return_tensors='pt')['attention_mask']

# Tokenize input labels
labels_encoded = [label2id[label] for label in labels]
labels_tensor = torch.tensor(labels_encoded)


# labels_tensor with DataLoader
dataset = torch.utils.data.TensorDataset(input_ids, attention_mask, labels_tensor)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
# Load pre-trained model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=77).to(device_name)

# Define optimizer with SGD
optimizer = SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

# Define learning rate scheduler
scheduler = MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)

# Define training parameters
num_epochs = 100
batch_size = 50

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device_name), attention_mask.to(device_name), labels.to(device_name)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    # Update learning rate
    scheduler.step()

    print(f"Epoch {epoch+1}, Loss: {total_loss}")

# Save the trained model
model.save_pretrained("path_to_save_model")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 500/500 [00:29<00:00, 17.20it/s]


Epoch 1, Loss: 135882.14225816727


Epoch 2: 100%|██████████| 500/500 [00:27<00:00, 18.00it/s]


Epoch 2, Loss: 159696.84820365906


...

Epoch 98, Loss: 168034.5842552185


Epoch 99: 100%|██████████| 500/500 [00:27<00:00, 18.26it/s]


Epoch 99, Loss: 163481.53737068176


Epoch 100: 100%|██████████| 500/500 [00:27<00:00, 18.19it/s]


Epoch 100, Loss: 175849.05043029785


In [28]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torch.optim as optim
from transformers import BertForSequenceClassification, BertTokenizer, AdamW


class NoiseDataset(Dataset):
    """ Dataset for loading softmax outputs and noise labels """
    def __init__(self, softmax_outputs, labels):
        self.softmax_outputs = softmax_outputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.softmax_outputs[idx], self.labels[idx]

# Placeholder for softmax probability extraction method
def extract_softmax_probabilities(model, dataloader, device):
    model.eval()
    softmax_probs = []
    with torch.no_grad():
        for batch in dataloader:
            batch = tuple(item.to(device) for item in batch)
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
            outputs = model(**inputs)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=-1)
            softmax_probs.append(probs.cpu().numpy())
    return np.vstack(softmax_probs)

# Assuming model_name, tokenizer, input_ids, attention_mask, labels_tensor, and device_name are defined
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(unique_labels))
model.to(device_name)

# Then, extract softmax probabilities from the trained model for Dn
softmax_probabilities = extract_softmax_probabilities(model, dataloader, device_name)

# Convert to tensor for NoiseDataset
softmax_probabilities_tensor = torch.tensor(softmax_probabilities, dtype=torch.float32)
noise_labels_tensor = torch.tensor(sampled_data['k'].values, dtype=torch.long)  # Assuming 'k' marks noise

# Prepare DataLoader for noise detector training
noise_dataset = NoiseDataset(softmax_probabilities_tensor, noise_labels_tensor)
noise_loader = DataLoader(noise_dataset, batch_size=64, shuffle=True)

# Noise Detector Model (simplified version)
class NoiseDetector(nn.Module):
    def __init__(self, input_size=77, hidden_size=50):
        super(NoiseDetector, self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)


    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize noise detector
number_of_classes = len(set(train_labels))
detector_model = NoiseDetector(input_size=number_of_classes)
detector_model.to(device_name)

# Training the noise detector
detector_optimizer = AdamW(detector_model.parameters(), lr=0.3)  # Use lr=0.03 for fine-tuning
criterion = nn.BCEWithLogitsLoss()

for epoch in range(10):
    for inputs, labels in noise_loader:
        inputs, labels = inputs.to(device_name), labels.to(device_name).float().unsqueeze(1)
        outputs = detector_model(inputs)
        loss = criterion(outputs, labels)

        detector_optimizer.zero_grad()
        loss.backward()
        detector_optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.4100641906261444
Epoch 2, Loss: 0.11801809072494507
Epoch 3, Loss: 0.20146818459033966
Epoch 4, Loss: 0.2693168818950653
Epoch 5, Loss: 0.3785228431224823
Epoch 6, Loss: 0.32573503255844116
Epoch 7, Loss: 0.38005587458610535
Epoch 8, Loss: 0.2172146886587143
Epoch 9, Loss: 0.3251130282878876
Epoch 10, Loss: 0.16406142711639404
